In [206]:
import pandas as pd
import seaborn as sns
import numpy as np
import re

from textwrap import wrap
from utils import *
from matplotlib import pyplot as plt
from pymongo import MongoClient
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn import preprocessing

In [198]:
acidentes_der = mongo_to_dataframe("acidentes_fatais_2008_2017")
acidentes_prf = mongo_to_dataframe("acidentes_rodovias_2010_2019") 

* Determinar causa de acidentes fatais em rodovias
    * Trabalho pode depender de perícia em certos casos

* Inputs:
    * condições climáticas
    * fase_dia, dia, mês
    * sentido_via (considerar)
    * uso_solo (considerar)
    * tipo_pista
    * tracado_pista
    * densidade da população
    * tipo_acidente
    * classificacao_acidente
    * pessoas
    * veiculos

### Preparação dos dados

In [134]:
def get_periodo(row, horario_column):
    if row[horario_column] == 'NAO_ESPECIFICADA':
        return 'NAO_ESPECIFICADA'
    elif wrap(row[horario_column], 2)[0] >= '06' and wrap(row[horario_column], 2)[0] < '12':
        return 'Amanhecer'
    elif wrap(row[horario_column] , 2)[0]>= '12' and wrap(row[horario_column] , 2)[0]< '18':
        return 'Pleno dia'
    elif wrap(row[horario_column] , 2)[0]>= '18' and wrap(row[horario_column] , 2)[0]< '19':
        return 'Anoitecer'
    elif wrap(row[horario_column] , 2)[0]>= '19' and wrap(row[horario_column] , 2)[0]<= '23':
        return 'Plena Noite'
    elif wrap(row[horario_column] , 2)[0]>= '00' and wrap(row[horario_column] , 2)[0]<= '06':
        return 'Madrugada'

In [234]:
dias = {
    'segunda-feira': 'Segunda',
    'terça-feira': 'Terça',
    'quarta-feira': 'Quarta',
    'quinta-feira': 'Quinta',
    'sexta-feira': 'Sexta',
    'sábado': 'Sábado',
    'domingo': 'Domingo'
}

uso_solo = {
    'Sim': 'Urbano',
    'Não': 'Rural'
}

tipos = {
    'Derramamento de carga': 'Outros',
    'Derramamento de Carga': 'Outros',
    'Danos Eventuais': 'Outros',
    'Danos eventuais': 'Outros',
    'Atropelamento de Animal': 'Atropelamento de animal',
    'Colisão com objeto móvel': 'Colisão com objeto em movimento',
    'Colisão com objeto estático': 'Colisão com objeto fixo',
    'Saída de leito carroçável': 'Saída de Pista'
}


causas = {
    'Sinalização da via insuficiente ou inadequada': 'Causas externas',
    'Fenômenos da natureza': 'Causas externas',
    'Mal súbito': 'Causas externas', 
    'Avarias e/ou desgaste excessivo no pneu': 'Desobediência de normas',  
    'Restrição de visibilidade': 'Causas externas', 
    'Agressão externa': 'Causas externas',
    'Carga excessiva e/ou mal acondicionada': 'Desobediência de normas', 
    'Ingestão de substâncias psicoativas': 'Desobediência de normas',
    'Objeto estático sobre o leito carroçável': 'Causas externas',  
    'Ingestão de álcool e/ou substâncias psicoativas pelo pedestre': 'Ingestão de álcool',
    'Sinalização da via insuficiente ou inadequada': 'Causas externas',   
    'Desobediência às normas de trânsito pelo pedestre' : 'Desobediência de normas', 
    'Deficiência ou não acionamento do sistema de iluminação/sinalização do veículo' : 'Desobediência de normas',
    'Defeito mecânico em veículo': 'Defeito mecânico no veículo',
    'Condutor dormindo' : 'Dormindo'
}

acidentes_prf = mongo_to_dataframe("acidentes_rodovias_2010_2019") 

columns_to_drop = ['data_inversa', 'horario', 'uf', 'ano', 'km', 'ignorados']

acidentes_prf.causa_acidente = acidentes_prf.causa_acidente.str.strip()
acidentes_prf.causa_acidente = acidentes_prf.causa_acidente.str.capitalize()
acidentes_prf.causa_acidente = acidentes_prf.causa_acidente.replace(causas)

acidentes_prf.horario = acidentes_prf.horario.str.strip()
acidentes_prf['fase_dia'] = acidentes_prf.apply(lambda row: get_periodo(row, 'horario'), axis = 1)

acidentes_prf.dia_semana = acidentes_prf.dia_semana.replace(dias)

acidentes_prf.uso_solo = acidentes_prf.uso_solo.str.strip()
acidentes_prf.uso_solo = acidentes_prf.uso_solo.replace(dias)

acidentes_prf.horario = pd.to_datetime(acidentes_prf.horario)
acidentes_prf['mes'] = acidentes_prf.horario.dt.month

acidentes_prf.condicao_metereologica = acidentes_prf.condicao_metereologica.str.strip()
acidentes_prf.condicao_metereologica = acidentes_prf.condicao_metereologica.replace({'Céu Claro': 'Ceu Claro'})

acidentes_prf.tipo_pista = acidentes_prf.tipo_pista.str.strip()
acidentes_prf.tracado_via = acidentes_prf.tracado_via.str.strip()

acidentes_prf.tipo_acidente = acidentes_prf.tipo_acidente.str.strip()
acidentes_prf.tipo_acidente = acidentes_prf.tipo_acidente.replace(tipos)

acidentes_prf.classificacao_acidente = acidentes_prf.classificacao_acidente.str.strip()

#acidentes_prf.km = acidentes_prf.km.astype(str)
#acidentes_prf.km = acidentes_prf.km.replace({',':'.'})

#acidentes_prf.br = acidentes_prf.br.astype(str)
acidentes_prf.br = acidentes_prf.br.replace({',':'.'})

acidentes_prf = acidentes_prf.drop(columns_to_drop, axis = 1)
acidentes_prf = acidentes_prf.dropna()

acidentes_prf.br = acidentes_prf.br.astype(float).astype(int)

acidentes_prf.sentido_via = acidentes_prf.sentido_via.str.strip()
#acidentes_prf.km = acidentes_prf.km.astype(float).astype(int)

In [207]:
acidentes_prf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11776 entries, 0 to 11780
Data columns (total 19 columns):
dia_semana                11776 non-null object
br                        11776 non-null int64
causa_acidente            11776 non-null object
tipo_acidente             11776 non-null object
classificacao_acidente    11776 non-null object
fase_dia                  11776 non-null object
sentido_via               11776 non-null object
condicao_metereologica    11776 non-null object
tipo_pista                11776 non-null object
tracado_via               11776 non-null object
uso_solo                  11776 non-null object
pessoas                   11776 non-null int64
mortos                    11776 non-null int64
feridos_leves             11776 non-null int64
feridos_graves            11776 non-null int64
ilesos                    11776 non-null int64
feridos                   11776 non-null int64
veiculos                  11776 non-null int64
mes                       11776 non

### Feature Selection

In [ ]:
svc = SVC(kernel="linear", C=1)


### SGD ()

#### Separação dos sets de treino e teste

In [235]:
le = preprocessing.LabelEncoder()

acidentes_prf.dia_semana = le.fit_transform(acidentes_prf.dia_semana)
acidentes_prf.causa_acidente = le.fit_transform(acidentes_prf.causa_acidente)
acidentes_prf.tipo_acidente = le.fit_transform(acidentes_prf.tipo_acidente)
acidentes_prf.classificacao_acidente = le.fit_transform(acidentes_prf.classificacao_acidente)
acidentes_prf.fase_dia = le.fit_transform(acidentes_prf.fase_dia)
acidentes_prf.sentido_via = le.fit_transform(acidentes_prf.sentido_via)
acidentes_prf.condicao_metereologica = le.fit_transform(acidentes_prf.condicao_metereologica)
acidentes_prf.tipo_pista = le.fit_transform(acidentes_prf.tipo_pista)
acidentes_prf.tracado_via = le.fit_transform(acidentes_prf.tracado_via)
acidentes_prf.uso_solo = le.fit_transform(acidentes_prf.uso_solo)
#for i in range(19):
    #acidentes_prf[:,i] = le.fit_transform(acidentes_prf[:,i])

In [236]:
X = acidentes_prf.drop('causa_acidente', axis = 1) 
y = (acidentes_prf['causa_acidente'])

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.3,
    random_state = 42,
    stratify = y
)

In [237]:
model = SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
       random_state=42, shuffle=True, tol=0.001, verbose=0, warm_start=False)

model.fit(X_train, y_train)

predicted = model.predict(X_test)

matrix = confusion_matrix(y_test, predicted)
print(matrix)

accuracy = model.score(X=X_test, y=y_test)
accuracy
# testar loss = modified hubber
# jobs
# penality
# random state



[[  0   0   0   0   0   0   0   0   6   0   0  10   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   7   0   0  16   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  46   0   0  54   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  10   0   0  13   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   5   0   0  12   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  65   0   0  24   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0  38   0   0  22   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  46   0   0  45   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0 638   0   0 435  11   0   0   0   1]
 [  0   0   0   0   0   0   0   0  12   1   0  10   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0 186   0   0 168   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 178   0   0 110   2   0   0   0   0]
 [  0   0   0   0   0   0   0   0 104   0   0 103  12   0   0   0   0]
 [  0   0   0   0   0   0   0   0 502   0   0 400   4   3   0   0   0]
 [  0 

0.2162468157373337

In [223]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [238]:
k_scores = []

for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)

    knn_score = knn.score(X_test, y_test)
    print('Score for k = ' + str(k) + ': ' + str(knn_score))
    
    k_scores.append(knn_score)

Score for k = 1: 0.2949334842909708
Score for k = 2: 0.2912538918765921
Score for k = 3: 0.31050099065949616
Score for k = 4: 0.3373903198414945
Score for k = 5: 0.3552221907727144
Score for k = 6: 0.36399660345315593
Score for k = 7: 0.3679592414378715
Score for k = 8: 0.3806962921030286
Score for k = 9: 0.3821115199547127
Score for k = 10: 0.3857911123690914


In [239]:
np_k_scores = np.asarray(k_scores)
best_k = np.argmax(np_k_scores) + 1
print('Best k to KNN Classifier: ' + str(best_k))

Best k to KNN Classifier: 10


In [240]:
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
confusion = confusion_matrix(y_test, y_pred, labels=)

In [226]:
acidentes_prf.causa_acidente.value_counts()

8     3617
13    3030
10    1178
11     968
12     729
16     613
2      332
7      307
5      299
6      200
14     108
9       81
1       78
3       77
4       56
0       52
15      51
Name: causa_acidente, dtype: int64